
> **AutoGen > Tutorial > Tool Use**
https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use

# Tool Use

이번 장에서는 에이전트에게 프로그래밍의 기능을 부여한 `code executors`**(코드 실행기)**에 대해 살펴봤다.
> **AutoGen > Tutorial > Tool Use**
https://velog.io/@heyggun/AutoGen-Code-Executors-UserProxyAgent-AssistantAgent

에이전트가 임의의 코드를 작성할 수 있다는 것은 유용하지만, 어떤 코드를 작성하게 할지 제어하는 것은 어려울 수 있다. 이때 **`Tool`** 이 등장한다.
`Tool`은 에이전트가 사용할 수 있는 미리 정의된 함수이다. 에이전트가 임의의 코드를 작성하는 대신, 툴을 호출하여 웹 검색, 계산 수행, 파일 읽기, 원격 API 호출 등의 작업을 수행할 수 있다. 어떤 툴을 에이전트에게 제공할지를 통제함으로써, 에이전트가 수행할 수 있는 작업도 제어할 수 있게 된다. 

**참고로 Tool 사용은 OpenAI 호환 툴 호출 API(OpenAI-compatible tool call API)를 지원하는 LLM에서만 가능하다**


## Creating Tools

`Tools`은 일반적인 python 함수로 만들 수 있다.
예를 들어 하나의 연산만 수행할 수 있는 계산기 툴을 아래와 같이 만들어 보자.


In [1]:
from typing import Annotated, Literal

In [2]:
Operator = Literal["+", "-", "*", "/"]

def calculator(a:int, b:int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a+b
    elif operator == "-":
        return a-b
    elif operator == "*":
        return a*b
    elif operator == "/":
        return int(a/b)
    else:
        raise ValueError("Invalid operator")

위 함수는 세 개의 인자를 받는다 `a`와 `b`는 연산에 사용될 정수이고, `operator`는 수행할 연산을 나타낸다. 여기서 **type hint**를 사용해서 인자와 반환 값의 타입을 정의했다.

> arguments(인자)와 return value(반환값)을 정의할 때는 항상 type hint(타입 힌트)를 사용하라. 타입 힌트는 툴의 사용법에 대해 에이저늩에게 유용한 정보를 제공한다.

## Registering Tools

툴을 만들었으면, 이제 대화에 참여하는 에이전트에게 그 툴을 `**register(등록)**` 할 수 있다.

In [4]:
import os
from autogen import ConversableAgent
from config import settings

api_key= settings.openai_api_key.get_secret_value()

In [5]:
llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key,
            }
        ]
}

In [9]:
assistant = ConversableAgent(
    name = "Assistant",
    system_message= """You are a helpful AI assistant. You can help with simple calculations.
Return 'TERMINATE' when the task is done""",
    llm_config = llm_config,
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg= lambda msg : msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode = "NEVER",
)

assistant.register_for_llm(name="calculator",
                           description="A simple calculator")(calculator)

user_proxy.register_for_execution(name="calculator")(calculator)